# `Associated Sports Events Feature`
This Jupyter Notebook will be used to parse the webpages and add data to the column in the master dataset which mentions related sports event based on date

In [29]:
import pandas as pd
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import csv
import warnings
warnings.filterwarnings('ignore')

Here we define a few functions, parse_link parses the webpages for the sports events tables, and saves those tables in the raw_files folder. The parse_dates function parses the string dates and converts them into date time objects showing the start and end datetime of a particular sports event. Finally, the save_formatted_sports_events_table brings both of these functions together and saves the formatted version.

In [30]:
def parse_link(url, year):
    # make a request to the webpage
    response = requests.get(url)

    # parse the HTML content using Beautiful Soup
    soup = BeautifulSoup(response.content, 'html.parser')

    # find the table containing the events
    table = soup.find('table', {'class': 'list'})

    # find all the rows in the table
    rows = table.find_all('tr')

    # create a CSV file to write the scraped data
    with open(f'raw_files/events_{year}.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)

        # write the table headers to the CSV file
        headers = [th.text.strip() for th in rows[0].find_all('th')]
        writer.writerow(headers)

        # write the table rows to the CSV file, filtering out rows containing "postponed" or "canceled"
        for row in rows[1:]:
            data = [td.text.strip() for td in row.find_all('td')]
            data = [x.replace('posponed', "postponed") for x in data]
            data = [x.replace('postoned', "postponed") for x in data]

            if 'postponed from' in data[0].lower():
                writer.writerow(data)
            elif 'canceled' in data[0].lower() or 'postponed' in data[0].lower():
                continue
            else:
                writer.writerow(data)


def parse_dates(date_str, year):
    date_str = date_str.split(' (')[0]
    date_str = date_str.replace('July', 'Jul')
    date_str = date_str.replace('June', 'Jun')
    date_str = date_str.replace('Sept', 'Sep')
    date_str = date_str.replace('–', '-')
    date_str = date_str.replace('8 Mar', 'Mar 8')
    date_str = date_str.replace('1 Mar', 'Mar 1')

    # split date range into start and end dates
    date_range = date_str.split('-')

    date_range = [x.strip() for x in date_range]

    if len(date_range) == 1:
        # if there is only one date, it is both the start and end date
        date_start = datetime.strptime(date_range[0] + f' {year}', '%b %d %Y')
        date_end = date_start
    else:
        # if second value can be converted to int, this is an individual month
        if len(date_range[-1]) <= 2:
            date_start = datetime.strptime(date_range[0] + f' {year}', '%b %d %Y')
            date_end = datetime(int(year), date_start.month, int(date_range[-1]))
        else:
            # otherwise, parse both start and end dates
            date_start = datetime.strptime(date_range[0] + f' {year}', '%b %d %Y')
            date_end = datetime.strptime(date_range[1] + f' {year}', '%b %d %Y')

    return date_start, date_end

def save_formatted_sports_events_table(url, year):
    parse_link(url, year)
    save_filename = f"raw_files/events_{year}.csv"
    df = pd.read_csv(save_filename)
    df['date_start'], df['date_end'] = zip(*df['Date(s)'].apply(parse_dates, year=year))
    df.to_csv(save_filename, index=False)

Now that we have the functions, we iterate over the years 2020, 2021, and 2022 and save the tables. We then merge these tables into one df, and save that csv.

In [31]:
years = [2020, 2021, 2022]
dfs = list()

for year in years:
    url = f'https://www.topendsports.com/events/calendar-{year}.htm'
    save_formatted_sports_events_table(url, year)

for year in years:
    df = pd.read_csv(f"raw_files/events_{year}.csv")
    dfs.append(df)

total = pd.concat(dfs)
total.drop(columns='Date(s)', inplace=True)
total.to_csv('raw_files/sports_events.csv', index=False)

We now have a fully merged csv with everything we need. We need to convert this into a df which has dates ranging from Jan 1st 2020 to Dec 31st 2022, which we will further merge with our master dataset.

In [32]:
df = pd.read_csv("raw_files/sports_events.csv")
# convert start and end times to datetime
df['date_start'] = pd.to_datetime(df['date_start'])
df['date_end'] = pd.to_datetime(df['date_end'])

# create a range of dates from Jan 1, 2020 to Dec 31, 2022
dates = pd.date_range(start='2020-01-01', end='2022-12-31')

# create an empty dataframe to store the results
result = pd.DataFrame(columns=['Date', 'Event'])

# iterate over each date and find the events that occurred on that date
for date in dates:
    events = df.loc[(df['date_start'] <= date) & (df['date_end'] >= date), 'Event'].tolist()
    event_str = ', '.join(events)
    result = result.append({'Date': date, 'Event': event_str}, ignore_index=True)

# print the resulting dataframe
result.to_csv("sports_events_final.csv", index=False)

We got our df to merge. We finally perform the merging.

In [33]:
df1 = pd.read_csv('../Master Datasets/Master_Dataset_Raw.csv')
df2 = pd.read_csv("sports_events_final.csv")

og_dates = df1['Account Created Date']

# Convert "Account Created Date" column in df1 to datetime
df1['Account Created Date'] = pd.to_datetime(df1['Account Created Date'])

# Convert "Date" column in df2 to datetime
df2['Date'] = pd.to_datetime(df2['Date'])

# merge the two dataframes based on the dates
merged_df = pd.merge(df1, df2[['Date', 'Event']], how='left', left_on=df1['Account Created Date'].dt.date, right_on=df2['Date'].dt.date)

# drop the duplicate date column
merged_df = merged_df.drop('key_0', axis=1)

# rename the 'Event' column to 'Events'
merged_df = merged_df.rename(columns={'Event': 'Events'})

# set the 'Events' column to NaN where there is no matching date
merged_df.loc[merged_df['Events'].isna(), 'Events'] = ''

# drop the duplicate date column
merged_df = merged_df.drop('Date', axis=1)

merged_df = merged_df.rename(columns={'Events': 'Sports Events'})

# display the resulting dataframe
merged_df

,Story Primary ID,Story ID,User Primary ID,User ID,Gender,Age,Title,Narrative,Media,Account Created Date,Date (No Timestamp),Interest,Sports Events
0,121169,STY1659426957,103,USR1606807023,female,34,Trend of the Year: Barbiecore,"The colour of the year is here, and it's *drum...",https://image.pixstory.com/Pixstory-image-1659...,2020-01-12 12:47:00,2020-01-12,"trends, fashion, barbie",Winter Youth Olympics
1,127727,STY1660634861,103,USR1606807023,female,34,Abomination of the day,We Indians do love to bastardise our foods- Ch...,https://image.pixstory.com/Pixstory-image-1660...,2020-01-12 12:47:00,2020-01-12,"Food, momos, weird menus",Winter Youth Olympics
2,123665,STY1660027898,103,USR1606807023,female,34,Shameful headline in 2022,Can professors not have personal lives? \n\nAd...,https://image.pixstory.com/Pixstory-image-1660...,2020-01-12 12:47:00,2020-01-12,"misogyny, st xaviers",Winter Youth Olympics
3,130517,STY1661151635,103,USR1606807023,female,34,Woman lawyer arrested for abusing security guard,"She was recorded on video manhandling him, sho...",https://image.pixstory.com/Pixstory-image-1661...,2020-01-12 12:47:00,2020-01-12,"Technology, History, Food, Entertainment, Spor...",Winter Youth Olympics
4,125466,STY1660372361,109,USR1606851217,others,31,What is the Inflation Reduction Act?,The House passed the Inflation Reduction Act o...,https://image.pixstory.com/Pixstory-image-1660...,2020-02-12 01:03:00,2020-02-12,us government,Six Nations
...,...,...,...,...,...,...,...,...,...,...,...,...,...
94995,125349,STY1660325841,71665,USR1660325291,NaN,18,COLLEGE LIFE,MODR CLASS,https://image.pixstory.com/Pixstory-image-1660...,2022-12-08 22:58:00,2022-12-08,"college life, college life umang","FIFA World Cup, World Championships"
94996,125345,STY1660325638,71673,USR1660325486,NaN,17,The one with the bestie😍,College life without a friend is useless. Enjo...,https://image.pixstory.com/Pixstory-image-1660...,2022-12-08 23:01:00,2022-12-08,college life,"FIFA World Cup, World Championships"
94997,125355,STY1660326971,71697,USR1660326022,NaN,19,College,"Sheridan college, HazelMcCallion Canpus",https://image.pixstory.com/Pixstory-image-1660...,2022-12-08 23:10:00,2022-12-08,"college life, college life umang","FIFA World Cup, World Championships"
94998,125351,STY1660326276,71698,USR1660326026,NaN,21,A tale of friendship,Check this out.,https://image.pixstory.com/Pixstory-image-1660...,2022-12-08 23:10:00,2022-12-08,friends,"FIFA World Cup, World Championships"


In [34]:
merged_df.to_csv('../Master Datasets/Master_Dataset_Raw_SportsEvents.csv', index=False)